### This script prepares the data of Burscher, Vliegenthart & De Vreese (2015) for ML

In [267]:
import pandas as pd

In [268]:
path_to_data ='/home/anne/tmpanne/'
df = pd.read_pickle(path_to_data + "news_dataset_annotated.pkl")

In [269]:
def recode_cat(x):
    if x == " ":
        x = '99'
    return x

df['v9_major'] = df['v9_major'].map(recode_cat)

In [270]:
df['main_topic'] = df['v9_major']
df['main_topic'] = df['main_topic'].astype(float)

In [271]:
# some weird values occur in the column 'v9_major'. Consequently, I did my best to map the values to their 'major' category. 

d = {205: 20,
729: 7,
192 : 19,
1903 : 19,
18 : 99,
21 : 99,
23 : 99,
28 : 99,
29 : 99, 
30: 99,
31 : 99,
27  : 99,
32 : 99, 
24: 20,
205 : 20, 
185 : 18,
93: 9,
1609 : 16,
72 : 7,
1916 : 19,
729 : 7,
185 : 99,
93 : 9,
1609 : 16,
72 : 7,
1916 : 19,
993 : 99,
311: 99}

In [272]:
# apply dictionarly to recode
df['main_topic'] = df['main_topic'].map(d)
df.main_topic.fillna(df.v9_major, inplace =True)
df['main_topic'] = df['main_topic'].astype(int)
df['main_topic'] = df['main_topic'].astype(str)

In [273]:
len(df['main_topic'].unique())

19

In [274]:
def label_topic(row):
    if row['main_topic'] == '99':
        return 'Overige'
    if row['main_topic'] == '1':
        return 'Macro-economie en belastingen'
    if row['main_topic'] == '2':
        return 'Burgerrechten en vrijheden'
    if row['main_topic'] == '3':
        return 'Gezondheid'
    if row['main_topic'] == '4':
        return 'Landbouw en Visserij'
    if row['main_topic'] == '5':
        return 'Arbeid'
    if row['main_topic'] == '6':
        return 'Onderwijs'
    if row['main_topic'] == '7':
        return 'Milieu'
    if row['main_topic'] == '8':
        return 'Energiebeleid'
    if row['main_topic'] == '9':
        return 'Immigratie en integratie'
    if row['main_topic'] == '10':
        return 'Verkeer en vervoer'
    if row['main_topic'] == '11':
        return 'Unkown'
    if row['main_topic'] == '12':
        return 'Justitie, Rechtspraak, Criminaliteit'
    if row['main_topic'] == '13':
        return 'sociale Zaken'
    if row['main_topic'] == '14':
        return 'Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning'
    if row['main_topic'] == '15':
        return 'Ondernemingen, Bankwezen en binnenlandse handel '
    if row['main_topic'] == '16':
        return 'Defensie'
    if row['main_topic'] == '17':
        return 'Wetenschappelijk onderzoek, technologie en communicatie'
    if row['main_topic'] == '18':
        return 'Buitenlandse handel'
    if row['main_topic'] == '19':
        return 'Buitenlandse zaken en ontwikkelingssamenwerking'
    if row['main_topic'] == '20':
        return 'Functioneren democratie en openbaar bestuur'
    if row['main_topic'] == '21':
        return 'Ruimtelijke ordening, publiek natuur- en waterbeheer'
    if row['main_topic'] == '22':
        return 'Unkown 2'
    if row['main_topic'] == '23':
        return 'Kunst, cultuur en entertainment'
    if row['main_topic'] == '00':
        return 'Toegevoegde codes voor media'
    
#df.apply(lambda row: label_topic(row),axis=1)
df['main_topic_label'] = df.apply (lambda row: label_topic (row),axis=1)
df[['main_topic','main_topic_label']][0:5]

,main_topic,main_topic_label
0,99,Overige
1,16,Defensie
2,16,Defensie
3,5,Arbeid
4,3,Gezondheid


In [275]:
df['main_topic'] = df['main_topic'].astype(str)
df['n'] = 1

# number of observations per topic:
agg = df.groupby(['main_topic', 'main_topic_label']).agg({'n': sum})
agg.sort_index()

,,n
main_topic,main_topic_label,
1,Macro-economie en belastingen,413
10,Verkeer en vervoer,371
12,"Justitie, Rechtspraak, Criminaliteit",1201
13,sociale Zaken,115
14,"Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning",114
15,"Ondernemingen, Bankwezen en binnenlandse handel",556
16,Defensie,394
17,"Wetenschappelijk onderzoek, technologie en communicatie",427
19,Buitenlandse zaken en ontwikkelingssamenwerking,1109


In [280]:
df['preprocessed_text'] = df.Processed_text.apply(', '.join)
len(df)

12559

In [285]:
"data/data_Wejgijge.pkl".split('/')[-1]

'data_Wejgijge.pkl'

In [286]:
# length of the datafile of Vermeer

len(pd.read_pickle('/home/anne/tmpanne/AEM_data/dataset_vermeer.pkl'))

3486

In [265]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier

X_train, X_test, y_train, y_test = train_test_split(df['Joined'], df['main_topic'], test_size=0.2, random_state=42)

pa = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='elasticnet', tol=1e-4, alpha=1e-6, max_iter=5000, random_state=42))
                            ])

pa.fit(X_train, y_train)
preds = pa.predict(X_test)

In [ ]:
print(classification_report(y_test, preds))

In [266]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, preds))

from sklearn.metrics import precision_recall_fscore_support as score

precision,recall,fscore,support=score(y_test, preds, average='weighted')
print ('Precision : {}'.format(precision))
print ('Recall    : {}'.format(recall))
print( 'F-score   : {}'.format(fscore))
print( 'Support   : {}'.format(support))

             precision    recall  f1-score   support

          1       0.61      0.33      0.43        90
         10       0.57      0.67      0.62        86
         12       0.62      0.66      0.64       248
         13       0.67      0.10      0.17        21
         14       0.70      0.28      0.40        25
         15       0.54      0.66      0.60        92
         16       0.54      0.40      0.46        81
         17       0.49      0.44      0.47        81
         19       0.56      0.70      0.62       242
          2       0.27      0.08      0.13        71
         20       0.55      0.64      0.59       251
          3       0.54      0.65      0.59        79
          4       0.55      0.81      0.65        21
          5       0.30      0.35      0.32        34
          6       0.74      0.76      0.75        37
          7       0.45      0.13      0.20        39
          8       0.64      0.50      0.56        14
          9       0.68      0.66      0.67   